# Dependências

In [ ]:
# Install required Google Cloud packages (commented out as these are typically one-time setup commands)
!pip install gcloud
!gcloud auth application-default login

# Import necessary Python libraries
import pandas as pd                # Data manipulation and analysis
import numpy as np                 # Numerical computing
import time                        # Time-related functions
import os                          # Operating system interfaces
import pandas_gbq                  # Pandas integration with BigQuery
from google.cloud import bigquery  # BigQuery client library
import glob                        # File path pattern matching
import openpyxl                    # Excel file handling
import csv                         # CSV file handling
import re                          # Regular expressions

# Note: The actual imports remain exactly as in the original code

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=97fe6d66d2ceb761c1af57c38604c8745aa17c16c04dee5a8bf37215a27b92cc
  Stored in directory: /root/.cache/pip/wheels/2a/62/75/3d74209bfebb8805823ae74afa28653aa1ea76d8b5a9d741ff
Successfully built gcloud
Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=x3WgNZsN2tISxeSLBCXNX5REiGn4XX&prompt=consent&token_usage=remote&access_type=offline&code_chal

# Tratamento

In [ ]:
# Import necessary libraries
import pandas as pd
import pandas_gbq
import re

# Define the SQL query to be executed in Google BigQuery.
# This query combines retiree data from multiple tables (2020-2024) using UNION ALL.
# For each record, it selects the year, month, and retirement type.
# It uses a CASE statement to select the superior organization ('ORGSUP_LOTACAO') if available,
# otherwise, it falls back to the specific organization ('ORG_LOTACAO').
# It also adds a static 'regime_juridico' column and a 'total' column (with a value of 1) for counting.
query = """
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2020_nov_siape_aposentados_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2021_nov_siape_aposentados_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2022_nov_siape_aposentados_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2023_nov_siape_aposentados_cadastro`
            UNION ALL
        SELECT ANO as ano, MES as mes, TIPO_APOSENTADORIA as tipo_aposentadoria,
            CASE WHEN ORGSUP_LOTACAO ="Sem informação" THEN ORG_LOTACAO ELSE ORGSUP_LOTACAO
            END as org_lotacao, 'REGIME JURIDICO UNICO' as regime_juridico,
            1 as total FROM `repositoriodedadosgpsp.portal_transparencia_cgu.2024_11_siape_aposentados_cadastro`
        """

# Execute the query using pandas_gbq.read_gbq and load the result into a pandas DataFrame called 'df'.
df = pandas_gbq.read_gbq(query, project_id='repositoriodedadosgpsp')

# Define a function to rename and format retirement types.
# It uses regular expressions (re.sub) to replace uppercase strings with properly cased and accented versions.
def renomear(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INVALIDEZ\b", "Aposentadoria por invalidez", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA VOLUNTARIA\b", "Aposentadoria voluntária", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA COMPULSORIA\b", "Aposentadoria compulsória", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bAPOSENTADORIA POR INCAPACIDADE\b", "Aposentadoria por incapacidade", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bOUTROS\b", "Outros", var_aposentadoria)
    return var_aposentadoria

# Apply the 'renomear' function to each element in the 'tipo_aposentadoria' column to standardize its values.
df['tipo_aposentadoria'] = df['tipo_aposentadoria'].apply(renomear)

# Convert the 'org_lotacao' column to title case (e.g., "MINISTRY OF HEALTH" becomes "Ministry Of Health").
df['org_lotacao'] = df['org_lotacao'].str.title()

# Define a second function to perform more specific renaming and corrections on organization names.
# This function standardizes ministry names, expands abbreviations, and corrects capitalization.
def renomear2(var_aposentadoria):
    var_aposentadoria = re.sub(r"\bMin Gestao E Inov Em Serv Publicos\b", "Ministério Gestão E Inovação Em Serviços Públicos", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Dos Povos Indigenas\b", "Ministério dos Povos Indígenas", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Da Previdencia Social\b", "Ministério Da Previdência Social", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio Do Planejamento E Orcamento\b", "Ministério Do Planejamento E Orçamento", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Da Integ E Do Desenv Regional\b", "Ministério da Integração e do Desenvolvimento Regional", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Do Desenv Agr E Agric Familiar\b", "Ministério do Desenvolvimento Agrário e Agricultura Familiar", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMin Desenvolv Ind Comercio E Servicos\b", "Ministério do Desenvolvimento, Indústria, Comércio e Serviços", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bComunicacoes\b", "Comunicações", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bMinisterio\b", "Ministério", var_aposentadoria)
    var_aposentadoria = re.sub(r"\bPrevidencia\b", "Previdência", var_aposentadoria)
    return var_aposentadoria

# Apply the 'renomear2' function to the 'org_lotacao' column for further cleaning and standardization.
df['org_lotacao'] = df['org_lotacao'].apply(renomear2)

# Define a third function to format the 'regime_juridico' column.
def renomear3(var_regime):
    var_regime = re.sub(r"\bREGIME JURIDICO UNICO\b", "Regime Jurídico Único", var_regime)
    return var_regime

# Apply the 'renomear3' function to the 'regime_juridico' column.
df['regime_juridico'] = df['regime_juridico'].apply(renomear3)

# Create a dictionary to map specific ministry names (keys) to broader thematic categories (values).
ministerios = {
    "Ministério Do Turismo": "Cultura, Turismo e Esportes",
    "Ministério Da Economia": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Da Saúde": "Saúde",
    "Ministério Da Justiça E Segurança Pública": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Da Educação": "Educação e Ciência e Tecnologia",
    "Ministério De Minas E Energia": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Defesa": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Da Ciência, Tecnologia, Inovações E Comunicações": "Educação e Ciência e Tecnologia",
    "Ministério Da Infraestrutura": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Presidência Da República": "Presidência Da República",
    "Ministério Das Relações Exteriores": "Relações Exteriores",
    "Ministério Da Agricultura E Pecuaria": "Meio ambiente, Agricultura e Agropecuária",
    "Ministério Da Agricultura, Pecuária E Abastecimento": "Meio ambiente, Agricultura e Agropecuária",
    "Ministério Do Desenvolvimento Regional": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Do Meio Ambiente": "Meio ambiente, Agricultura e Agropecuária",
    "Controladoria-Geral Da União": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Min Desenv Assis Soci Famil Combate Fome": "Desenvolvimento Social e Direitos Humanos",
    "Ministério Do Desenvolvimento Social": "Desenvolvimento Social e Direitos Humanos",
    "Defensoria Pública Da União": "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Conselho Nacional De Desenvolvimento Científico E Tecnológico": "Educação e Ciência e Tecnologia",
    "Ministério Gestão E Inovação Em Serviços Públicos": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Da Cultura": "Cultura, Turismo e Esportes",
    "Ministério Dos Transportes": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Previdência Social": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Do Planejamento E Orçamento": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério da Integração e do Desenvolvimento Regional": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério do Desenvolvimento Agrário e Agricultura Familiar": "Meio ambiente, Agricultura e Agropecuária",
    "Ministério dos Povos Indígenas": "Desenvolvimento Social e Direitos Humanos",
    "Ministério Do Trabalho E Emprego": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério De Portos E Aeroportos": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério do Desenvolvimento, Indústria, Comércio e Serviços": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Da Fazenda": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Das Comunicações": "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Caixa De Financiamento Imobiliário Da Aeronáutica": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Ministério Do Trabalho E Previdência": "Economia, Finanças, Planejamento, Previdência e Gestão",
    "Advocacia-Geral Da Uniao" : "Justiça, Segurança e Órgãos de Controle e Defesa",
    "Ministério Das Cidades" : "Infraestrutura, Transporte, Desenvolvimento Regional, Portos e Aeroportos, Minas e Energia",
    "Ministério Do Esporte" : "Cultura, Turismo e Esportes",
    "Ministério Gestão E Inovação Em Serviços Públicos": "Economia, Finanças, Planejamento, Previdência e Gestão",
}

# Define a function that takes an organization name and returns its category from the dictionary.
def categorizando(x):
    # Check if the organization name exists as a key in the 'ministerios' dictionary.
    if x in ministerios:
        # If it exists, return the corresponding category value.
        return ministerios[x]

# Create a new column 'categoria_ministerios' by applying the 'categorizando' function
# to each organization name in the 'org_lotacao' column.
df['categoria_ministerios'] = df['org_lotacao'].apply(categorizando)

# Upload

In [ ]:
# Import the bigquery library from google.cloud
from google.cloud import bigquery

# Define the schema for the destination BigQuery table.
# The schema is a list of SchemaField objects, where each object defines a column's:
# 1. Name (e.g., 'ano')
# 2. Data type (e.g., 'INTEGER')
# 3. Description (e.g., 'Ano de referência da observação')
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('mes','INTEGER',description='Mês de referência da observação'),
        bigquery.SchemaField('tipo_aposentadoria','STRING',description='Tipo de aposentadoria'),
        bigquery.SchemaField('org_lotacao','STRING',description='Órgão de lotação'),
        bigquery.SchemaField('regime_juridico','STRING',description='Regime jurídico'),
        bigquery.SchemaField('total','INTEGER',description='Quantidade total da observação'),
        bigquery.SchemaField('categoria_ministerios','STRING',description='Categorias dos ministérios')]

# Initialize the BigQuery client, specifying the Google Cloud project ID.
# This client object is the main entry point for interacting with the BigQuery API.
client = bigquery.Client(project='repositoriodedadosgpsp')

# Create a reference to the BigQuery dataset named 'perfil_remuneracao'.
# This object points to the dataset where the table will be created or updated.
dataset_ref = client.dataset('perfil_remuneracao')

# Create a reference to the target table within the dataset specified earlier.
# The table will be named 'SIAPE_aposentadorias_v4'.
table_ref = dataset_ref.table('SIAPE_aposentadorias_v4')

# Configure the load job by creating a LoadJobConfig object.
# Here, we specify the schema that BigQuery should use for the table. This ensures
# that the columns in BigQuery have the correct data types and descriptions.
job_config = bigquery.LoadJobConfig(schema=schema)

# Start the job to load data from the pandas DataFrame 'df' into the specified BigQuery table ('table_ref').
# The job is configured with the previously defined 'job_config'. This command sends the data to BigQuery.
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)

# Wait for the load job to complete and retrieve its result.
# This line is blocking and will pause the script's execution until the data upload is finished.
# It's crucial for ensuring the data is fully loaded before the script ends or proceeds.
job.result()